In [12]:
import torch
import wandb
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch.nn.functional as F
from torchvision import transforms
from Utilities import Utilities as Util

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using mps device


In [13]:
# Basic Function definitions

def train(dataloader, model, loss_fn, optimizer, epoch, logcount=5, wandb_log=False):
    size = len(dataloader.dataset)
    loginterval = len(dataloader) // logcount
    average_loss = 0.0

    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        average_loss += loss.detach().item()
        
        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if (batch % loginterval == 0) and (batch > 0):
            log_loss = average_loss / loginterval
            average_loss = 0
            current = batch * len(X)
            print(f"loss: {log_loss:>8f}  [{current:>5d}/{size:>5d}]")
            if wandb_log:
                wandb.log({"epoch": epoch, "train_loss": log_loss})

def test(dataloader, model, loss_fn, epoch, wandb_log=False):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).detach().item()
    test_loss /= num_batches
    if wandb_log:
        wandb.log({"epoch" : epoch, "test_loss": test_loss})
    print(f"Test Error: \n Avg loss: {test_loss:>8f} \n")

def parameterCount(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

In [14]:
class NeuralNetwork(nn.Module):
    def __init__(self, inFilters, outFilters):
        super().__init__()

        self.convIn = nn.Conv2d(inFilters, outFilters, 1, padding=0)
        self.convCent = nn.ModuleList([nn.Conv2d(outFilters * 4, outFilters * 4, 1, padding=0) for _ in range(5)])
        self.convOut = nn.Conv2d(outFilters, outFilters, 1, padding=0)
        self.ReLU = nn.ReLU()
        self.Sigmoid = nn.Sigmoid()

    def forward(self, x):  
        x = self.convIn(x)
        x = self.ReLU(x)
        x = self.convOut(x)
        x = self.Sigmoid(x)
        
        return x

In [15]:
# Model Hyperparameters / Config

BatchSize = 2048
datasetPathV1 = "../../Datasets/HumanExamples/GeneratedDatasets/HD5,TS0.8,RULESETS(1-29),Legacy"
datasetPathV2 = "../../Datasets/HumanExamples/GeneratedDatasets/HD6,TS0.8,RULESETS(1-29)"

In [16]:
# Load datasets into memory

XtrainV1 = torch.from_numpy(np.fromfile(f'{datasetPathV1}/XTrain.bin', dtype=bool).astype(np.float32).reshape(-1, 11, 15, 15))
XtestV1 = torch.from_numpy(np.fromfile(f'{datasetPathV1}/XTest.bin', dtype=bool).astype(np.float32).reshape(-1, 11, 15, 15))


XtrainV2 = torch.from_numpy(np.fromfile(f'{datasetPathV2}/XTrain.bin', dtype=bool).astype(np.float32).reshape(-1, 7, 15, 15))
XtestV2 = torch.from_numpy(np.fromfile(f'{datasetPathV2}/XTest.bin', dtype=bool).astype(np.float32).reshape(-1, 7, 15, 15))

print(XtrainV1.shape)
print(XtrainV2.shape)

torch.Size([800000, 11, 15, 15])
torch.Size([800000, 7, 15, 15])


In [11]:
i = 0
d = 3
print("Old:")
print(Util.sliceLegacyGamestate(XtrainV1[i], 5, d))
print("New:")
print(Util.sliceGamestate(XtrainV2[i], 6, d))

Old:
     0   1   2   3   4   5   6   7   8   9  10  11  12  13  14
   --------------------------------------------------------------
14 |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |
   --------------------------------------------------------------
13 |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |
   --------------------------------------------------------------
12 |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |
   --------------------------------------------------------------
11 |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |
   --------------------------------------------------------------
10 |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |
   --------------------------------------------------------------
 9 |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |
   --------------------------------------------------------------
 8 |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |
   ------------

In [7]:
for i in range(80000):
    for y in range(2):
        out = Util.sliceLegacyGamestate(XtrainV1[i], 5, y)
        inp = Util.sliceGamestate(XtrainV2[i], 6, y)
        if out != inp:
            print(f'on: {i}')
            print(out)
            print(inp)
            break
    else:
        break

on: 0
     0   1   2   3   4   5   6   7   8   9  10  11  12  13  14
   --------------------------------------------------------------
14 |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |
   --------------------------------------------------------------
13 |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |
   --------------------------------------------------------------
12 |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |
   --------------------------------------------------------------
11 |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |
   --------------------------------------------------------------
10 |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |
   --------------------------------------------------------------
 9 |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |
   --------------------------------------------------------------
 8 |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |
   -----------

In [17]:
train_dataset = TensorDataset(XtrainV2, XtrainV1)
train_loader = DataLoader(train_dataset, batch_size=BatchSize, shuffle=True)
test_dataset = TensorDataset(XtestV2, XtestV1)
test_loader = DataLoader(test_dataset, batch_size=BatchSize, shuffle=False)

In [8]:
model = NeuralNetwork(7, 11).to(device)
#model = torch.compile(model)

wandb_logging = False
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters())

paramCount = sum(p.numel() for p in model.parameters())
print(paramCount)

10747


In [9]:
epochs = 15
for epoch in range(epochs):
    print(f"Epoch {epoch+1}\n-------------------------------")
    train(train_loader, model, loss_fn, optimizer, epoch, logcount=10, wandb_log=wandb_logging)
    test(test_loader, model, loss_fn, epoch, wandb_log=wandb_logging)
print("Done!")

Epoch 1
-------------------------------
loss: 2.693623  [79872/800000]
loss: 2.283614  [159744/800000]
loss: 2.225782  [239616/800000]
loss: 2.222298  [319488/800000]
loss: 2.221176  [399360/800000]
loss: 2.219337  [479232/800000]
loss: 2.224290  [559104/800000]
loss: 2.199594  [638976/800000]
loss: 2.217031  [718848/800000]
loss: 2.210810  [499200/800000]
Test Error: 
 Avg loss: 2.006967 

Epoch 2
-------------------------------
loss: 2.266529  [79872/800000]
loss: 2.217804  [159744/800000]
loss: 2.205307  [239616/800000]
loss: 2.213696  [319488/800000]
loss: 2.207920  [399360/800000]
loss: 2.217217  [479232/800000]
loss: 2.203628  [559104/800000]
loss: 2.212611  [638976/800000]
loss: 2.209015  [718848/800000]
loss: 2.209503  [499200/800000]
Test Error: 
 Avg loss: 2.006766 

Epoch 3
-------------------------------
loss: 2.265973  [79872/800000]
loss: 2.213997  [159744/800000]
loss: 2.208335  [239616/800000]
loss: 2.220592  [319488/800000]
loss: 2.210577  [399360/800000]
loss: 2.20927